In [1]:
library(rstan)
library(rstansensitivity)

library(ggplot2)
library(dplyr)
library(reshape2)

rstan_options(auto_write=TRUE)

# Set this to be the appropriate location of the repo on your computer.
# Run from anywhere in the StanSensitivity repository.
git_repo <- system("git rev-parse --show-toplevel", intern=TRUE)
example_dir <- file.path(git_repo, "examples/schools/")
model_name <- GenerateSensitivityFromModel(
  file.path(example_dir, "models/schools-1.stan"))

Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.17.3, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)
Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: reshape2


In [ ]:
model <- stan_model(GetSamplingModelFilename(model_name))

# Load the data and hyperparameters.
stan_data <- new.env()
source(paste(example_dir, "schools-1.data.R", sep=""), local=stan_data)
stan_data <- as.list(stan_data)

stan_sensitivity_list <- GetStanSensitivityModel(model_name, stan_data)
sens_result <- GetStanSensitivityFromModelFit(sampling_result, stan_sensitivity_list)

In file included from /usr/local/lib/R/site-library/BH/include/boost/config.hpp:39:0,
                 from /usr/local/lib/R/site-library/BH/include/boost/math/tools/config.hpp:13,
                 from /usr/local/lib/R/site-library/StanHeaders/include/stan/math/rev/core/var.hpp:7,
                 from /usr/local/lib/R/site-library/StanHeaders/include/stan/math/rev/core/gevv_vvv_vari.hpp:5,
                 from /usr/local/lib/R/site-library/StanHeaders/include/stan/math/rev/core.hpp:12,
                 from /usr/local/lib/R/site-library/StanHeaders/include/stan/math/rev/mat.hpp:4,
                 from /usr/local/lib/R/site-library/StanHeaders/include/stan/math.hpp:4,
                 from /usr/local/lib/R/site-library/StanHeaders/include/src/stan/model/model_header.hpp:4,
                 from file279c376a013c.cpp:8:
/usr/local/lib/R/site-library/BH/include/boost/config/compiler/gcc.hpp:186:0: warning: "BOOST_NO_CXX11_RVALUE_REFERENCES" redefined
 #  define BOOST_NO_CXX11_RVALUE_RE

In [ ]:
num_samples <- 5000
sampling_file <- paste(model_name, "_sampling.Rdata", sep="")
if (!file.exists(sampling_file)) {
  print("Running sampler.")
  sampling_time <- time.time()
  sampling_result <-
    sampling(model, data=stan_data, chains=1, iter=num_samples)
  sampling_time <- time.time() - sampling_time
  save(sampling_result, sampling_result, sampling_time,
       file=sampling_file)
} else {
  print(sprintf("Loading cached samples from %s", sampling_file))
  load(sampling_file)  
}

In [ ]:
print(summary(sampling_result))

tidy_results <- GetTidyResult(sens_result)
PlotSensitivities(filter(tidy_results, abs(normalized_sensitivity)  > 1.0))
PlotSensitivities(filter(tidy_results, grepl("beta", parameter)))

In [ ]:
perturb_par <- "R.3.3"
epsilon <- 0.1

# Check that the perturbation is big enough that we expect to produce
# a measurable difference in the output.
se_mean <- summary(sampling_result)$summary[, "se_mean"]
min_epsilon <- 2.0 * min(se_mean / abs(sens_result$sens_mat[perturb_par, ]))
if (epsilon < min_epsilon) {
  warning("The expected change is less than twice the mean standard error for every parameter.")
}


In [ ]:
# Re-run MCMC
stan_data_perturb <- stan_data
stan_data_perturb[["R"]][3, 3] <- stan_data_perturb[["R"]][3, 3] + epsilon
perturbed_sampling_file <- paste(model_name, "_perturbed_sampling.Rdata", sep="")
if (!file.exists(perturbed_sampling_file)) {
  print("Running sampler.")
  sampling_time_perturb <- time.time()
  sampling_result_perturb <- sampling(model, data=stan_data_perturb, chains=1,
                                      iter=num_samples)
  sampling_time_perturb <- time.time() - sampling_time_perturb
  save(sampling_result_perturb, stan_data_perturb, sampling_time_perturb,
       file=perturbed_sampling_file)
} else {
  print(sprintf("Loading cached perturbed samples from %s", perturbed_sampling_file))
  load(perturbed_sampling_file)  
}